In [2]:
import pickle

inputs = pickle.load(open('/home/vlai-vqa-nle/minhtq/vqa-nle/tests/debug_inputs.pkl', 'rb'))
rewards = pickle.load(open('/home/vlai-vqa-nle/minhtq/vqa-nle/tests/debug_rewards.pkl', 'rb'))

print("INPUTS:")
print(f"Type: {type(inputs)}")
print(f"Length: {len(inputs)}")

# Xem item đầu tiên
if len(inputs) > 0:
    print(f"\nFirst item type: {type(inputs[0])}")
    
    # Nếu item là dict
    if isinstance(inputs[0], dict):
        print(f"Keys: {inputs[0].keys()}")
        for k, v in inputs[0].items():
            print(f"  {k}: {type(v)}")
            if hasattr(v, 'shape'):
                print(f"    Shape: {v.shape}")
            elif isinstance(v, (list, tuple)):
                print(f"    Length: {len(v)}")
            elif isinstance(v, str):
                print(f"    Value: {v[:100]}...")  # Show 100 ký tự đầu
    else:
        print(f"First item: {inputs[0]}")

print("\n" + "="*80)
print("REWARDS:")
print(f"Shape: {rewards.shape}")
print(rewards)


INPUTS:
Type: <class 'list'>
Length: 4

First item type: <class 'dict'>
Keys: dict_keys(['messages', 'images', 'solution', 'prompt_id', 'request_id', 'response_token_ids', 'finish_reason', 'is_truncated'])
  messages: <class 'list'>
    Length: 3
  images: <class 'list'>
    Length: 1
  solution: <class 'str'>
    Value: <answer>gas</answer><explain>đầu đốt gas đã được lắp đặt</explain>...
  prompt_id: <class 'str'>
    Value: prompt_0...
  request_id: <class 'str'>
    Value: chatcmpl-47b06c148ff446c59c68e541dd1b26d3...
  response_token_ids: <class 'list'>
    Length: 165
  finish_reason: <class 'str'>
    Value: stop...
  is_truncated: <class 'bool'>

REWARDS:
Shape: torch.Size([4, 3])
tensor([[1.0000, 0.1816, 0.8369],
        [1.0000, 0.1175, 0.6453],
        [1.0000, 1.0000, 0.3425],
        [1.0000, 1.0000, 0.6966]], device='cuda:0')


In [3]:
import pickle
import json

inputs = pickle.load(open('/home/vlai-vqa-nle/minhtq/vqa-nle/tests/debug_inputs.pkl', 'rb'))

# Convert sang JSON để xem dễ hơn (bỏ qua non-serializable objects)
def to_json_safe(obj):
    if isinstance(obj, (str, int, float, bool, type(None))):
        return obj
    elif isinstance(obj, (list, tuple)):
        return [to_json_safe(x) for x in obj]
    elif isinstance(obj, dict):
        return {k: to_json_safe(v) for k, v in obj.items()}
    else:
        return str(obj)

inputs_safe = to_json_safe(inputs)
print(json.dumps(inputs_safe, indent=2, ensure_ascii=False))


[
  {
    "messages": [
      {
        "role": "system",
        "content": "<image>Bạn là hệ thống Visual Question Answering (VQA). Nhiệm vụ của bạn là trả lời và giải thích các câu hỏi dựa trên nội dung của hình ảnh được cung cấp.",
        "loss": null
      },
      {
        "role": "user",
        "content": "Câu hỏi: Bếp sử dụng gas hay điện?\n    Vui lòng trả lời câu hỏi sau dựa trên hình ảnh. Hãy trả lời theo định dạng sau:\n    <REASONING>Quá trình suy luận chi tiết dẫn đến câu trả lời cuối cùng</REASONING>\n    <answer>Câu trả lời (một từ hoặc cụm từ ngắn)</answer>\n    <explain>Giải thích một câu ngắn gọn chứng minh câu trả lời</explain>",
        "loss": null
      },
      {
        "role": "assistant",
        "content": "<REASONING>Hình ảnh hiển thị một bếp điện.  Có thể thấy rõ bảng điều khiển của bếp điện với các nút bấm và màn hình hiển thị nhiệt độ, khác biệt với các chi tiết của bếp gas, chẳng hạn như nắp xả gas, ống dẫn khí và núm bật lửa.  Bếp điện được đặt trên

In [ ]:
import torch
from typing import List, Dict, Any, Tuple
import re
def filter_by_rpt(
    inputs: List[Dict],
    rewards: torch.Tensor,
    num_generations: int = 16,
    filter_top_k: int = 8,
    metric: str = "token_efficiency", 
    length_penalty: float = 0.01  # chỉ dùng khi metric="combined"
) -> Tuple[List[Dict], torch.Tensor, torch.Tensor]:
    """
    GFPO filtering theo paper Section 3.
    
    Args:
        inputs: List of completion dicts with 'messages' key
        rewards: Tensor of shape (N,) containing rewards
        num_generations: Number of completions per prompt (G)
        filter_top_k: Number to keep per prompt (k)
        metric: "length", "token_efficiency", or "combined"
        length_penalty: Penalty weight (only for "combined")
    
    Returns:
        filtered_inputs: List of all inputs (bao gồm cả rejected)
        filtered_rewards: Tensor of all rewards
        mask: Binary mask (1 = retained, 0 = rejected)
    """
    
    # Count tokens
    num_tokens = []
    for inp in inputs:
        completion = inp['messages'][-1]['content']
        reasoning_match = re.search(r"<REASONING>(.*?)</REASONING>", completion, flags=re.DOTALL | re.IGNORECASE)
        reasoning = reasoning_match.group(1).strip() if reasoning_match else ""

        tokens = reasoning.split()  
        print(f"Reasoning: {reasoning}")
        print(f"Tokens: {tokens}")
        num_tokens.append(len(tokens))
    
    num_tokens = torch.tensor(num_tokens, dtype=torch.float32, device=rewards.device)
    
    # Calculate metric scores based on chosen metric
    if metric == "length":
        # Shortest k/G: shorter is better (negate for topk)
        metric_scores = -num_tokens
    elif metric == "token_efficiency":
        # Token Efficiency: reward/length, higher is better
        metric_scores = rewards / num_tokens.clamp(min=1.0)
    elif metric == "combined":
        # Your approach: RPT - length penalty
        rpt_scores = rewards / num_tokens.clamp(min=1.0)
        metric_scores = rpt_scores - length_penalty * num_tokens
    else:
        raise ValueError(f"Unknown metric: {metric}")
    
    # Filter top-k per group (Algorithm 1 in paper)
    num_prompts = len(inputs) // num_generations
    mask = torch.zeros(len(inputs), dtype=torch.float32, device=rewards.device)
    
    for i in range(num_prompts):
        start = i * num_generations
        end = start + num_generations
        
        # Get scores for this prompt group
        group_scores = metric_scores[start:end]
        
        # Select top-k (REJECTIONSAMPLE in paper)
        k_actual = min(filter_top_k, len(group_scores))
        _, top_k_idx = torch.topk(group_scores, k=k_actual)
        
        # Set mask to 1 for retained responses
        global_indices = start + top_k_idx
        mask[global_indices] = 1.0
    
    # Compute statistics (μ_S, σ_S) ONLY on retained set S
    retained_indices = mask.bool()
    retained_rewards = rewards[retained_indices]
    
    mu_S = retained_rewards.mean()
    sigma_S = retained_rewards.std().clamp(min=1e-8)
    
    # Print info
    num_retained = int(mask.sum().item())
    print(f"GFPO Filtering ({metric}): {num_retained}/{len(inputs)} samples retained")
    print(f"  Retained rewards: mean={mu_S:.4f}, std={sigma_S:.4f}")
    print(f"  Length: mean={num_tokens[retained_indices].mean():.1f}, "
          f"all={num_tokens.mean():.1f}")
    if metric in ["token_efficiency", "combined"]:
        rpt = rewards / num_tokens.clamp(min=1.0)
        print(f"  RPT: retained={rpt[retained_indices].mean():.4f}, "
              f"all={rpt.mean():.4f}")
    
    return inputs, rewards, mask


# ============================================
# TEST CODE
# ============================================
if __name__ == '__main__':
    # Simple test data
    inputs = [
            # --- Prompt 1, Completion 1 (Đúng, ngắn gọn) ---
            {
                "messages": [
                    {"role": "system", "content": "Bạn là hệ thống VQA."},
                    {"role": "user", "content": "Bếp sử dụng gas hay điện?"},
                    {"role": "assistant", "content": "<REASONING>Hình ảnh cho thấy đầu đốt gas.</REASONING>\n<answer>Gas</answer>\n<explain>Bếp có đầu đốt đặc trưng của bếp gas.</explain>"}
                ],
                "solution": "<answer>gas</answer>"
            },
            # --- Prompt 1, Completion 2 (Sai, dài dòng) ---
            {
                "messages": [
                    {"role": "system", "content": "Bạn là hệ thống VQA."},
                    {"role": "user", "content": "Bếp sử dụng gas hay điện?"},
                    {"role": "assistant", "content": "<REASONING>Bề mặt bếp phẳng và có các nút điều khiển cảm ứng, đây là đặc điểm của bếp điện hiện đại. Không có kiềng hay đầu đốt của bếp gas.</REASONING>\n<answer>Điện</answer>\n<explain>Bề mặt phẳng và bảng điều khiển cảm ứng cho thấy đây là bếp điện.</explain>"}
                ],
                "solution": "<answer>gas</answer>"
            },
            # --- Prompt 1, Completion 3 (Đúng, nhưng dài) ---
            {
                "messages": [
                    {"role": "system", "content": "Bạn là hệ thống VQA."},
                    {"role": "user", "content": "Bếp sử dụng gas hay điện?"},
                    {"role": "assistant", "content": "<REASONING>Quan sát kỹ hình ảnh, tôi nhận thấy có các kiềng sắt và mâm chia lửa, đây là những bộ phận không thể thiếu của một chiếc bếp gas truyền thống. Mặc dù có lò vi sóng ở trên, bản thân bếp nấu chính vẫn là bếp gas.</REASONING>\n<answer>Gas</answer>\n<explain>Bếp có kiềng sắt và mâm chia lửa, chứng tỏ nó hoạt động bằng gas.</explain>"}
                ],
                "solution": "<answer>gas</answer>"
            },
            # --- Prompt 1, Completion 4 (Không chắc chắn) ---
            {
                "messages": [
                    {"role": "system", "content": "Bạn là hệ thống VQA."},
                    {"role": "user", "content": "Bếp sử dụng gas hay điện?"},
                    {"role": "assistant", "content": "<REASONING>Thông tin trong ảnh không đủ rõ để xác định. Bếp có vẻ hiện đại nhưng không loại trừ khả năng là bếp gas âm.</REASONING>\n<answer>Không thể khẳng định</answer>\n<explain>Không đủ chi tiết để phân biệt giữa bếp gas âm và bếp điện.</explain>"}
                ],
                "solution": "<answer>gas</answer>"
            }
        ]
        
    # Rewards tương ứng (đã aggregate)
    # 1.0: Đúng & Tốt
    # 0.0: Sai
    # 0.8: Đúng nhưng dài
    # 0.2: Không chắc chắn
    rewards = torch.tensor([1.0, 0.0, 0.8, 0.2])
        
    # --- Chạy test ---
    print("=== TRƯỚC KHI LỌC ===")
    print(f"Tổng số mẫu: {len(inputs)}")
    for i, inp in enumerate(inputs):
        content = inp['messages'][-1]['content'].split('\n')[1]
        print(f"[{i}] {content.strip()} - Reward: {rewards[i]:.1f}")

    print("\n=== ĐANG LỌC ===")
    filtered_inputs, filtered_rewards, mask = filter_by_rpt(
            inputs=inputs,
            rewards=rewards,
            num_generations=4, # 4 completions cho 1 prompt
            filter_top_k=2,      # Giữ lại 2 tốt nhất
        )

    print("\n=== SAU KHI LỌC ===")
    print(f"Số mẫu còn lại: {len(filtered_inputs)}")
    for i, inp in enumerate(filtered_inputs):
        content = inp['messages'][-1]['content'].split('\n')[1]
        print(f"[{i}] {content.strip()} - Reward: {filtered_rewards[i]:.1f} - Mask: {mask[i]:.1f}")


=== TRƯỚC KHI LỌC ===
Tổng số mẫu: 4
[0] <answer>Gas</answer> - Reward: 1.0
[1] <answer>Điện</answer> - Reward: 0.0
[2] <answer>Gas</answer> - Reward: 0.8
[3] <answer>Không thể khẳng định</answer> - Reward: 0.2

=== ĐANG LỌC ===
Reasoning: Hình ảnh cho thấy đầu đốt gas.
Tokens: ['Hình', 'ảnh', 'cho', 'thấy', 'đầu', 'đốt', 'gas.']
Reasoning: Bề mặt bếp phẳng và có các nút điều khiển cảm ứng, đây là đặc điểm của bếp điện hiện đại. Không có kiềng hay đầu đốt của bếp gas.
Tokens: ['Bề', 'mặt', 'bếp', 'phẳng', 'và', 'có', 'các', 'nút', 'điều', 'khiển', 'cảm', 'ứng,', 'đây', 'là', 'đặc', 'điểm', 'của', 'bếp', 'điện', 'hiện', 'đại.', 'Không', 'có', 'kiềng', 'hay', 'đầu', 'đốt', 'của', 'bếp', 'gas.']
Reasoning: Quan sát kỹ hình ảnh, tôi nhận thấy có các kiềng sắt và mâm chia lửa, đây là những bộ phận không thể thiếu của một chiếc bếp gas truyền thống. Mặc dù có lò vi sóng ở trên, bản thân bếp nấu chính vẫn là bếp gas.
Tokens: ['Quan', 'sát', 'kỹ', 'hình', 'ảnh,', 'tôi', 'nhận', 'thấy', 'có', '